In [1]:
import pandas as pd
import numpy as np

#all predictors
predictors = ['m_nmes', 
              'm_intrinsic_sugars',
              'm_starch', 
              'm_fiber', 
              'm_sucrose',
              'm_total_sugars']

#main manuscript predictors
predictors = ['m_nmes', 
              'm_intrinsic_sugars',
              'm_starch', 
              'm_fiber']

outcomes = ['bodyfat_percent','bmi', 'waistcirumference', 'trig_log', 'ldl', 'hdl',
       'hba1c_percent' ]


def determine_trend_symbol(data):
    """Determine trend symbol based on data trend"""
    lower = data['Contrast_CI_lower'].to_list()[0]
    upper = data['Contrast_CI_upper'].to_list()[0]

    if lower > 0 and upper > 0:
        return '↓'
    elif lower < 0 and upper < 0:
        return '↑'
    else:
        return '⌀'


In [2]:
#Redid this to make a way nicer table! - will have 4 tables in the end!

precision = 1

for ctr in range(1,4):
        
    df = pd.read_csv('../results/associations/association_table_model_'+str(ctr)+'.csv')

    all_rows = []
    for outcome in outcomes:
        row = [outcome]
        for predictor in predictors:
            sub_df = df.loc[(df['Outcome'] == outcome) & (df['Predictor'] == predictor)].reset_index(drop=True)
            trend_symbol = determine_trend_symbol(sub_df)
            for i in range(0, len(sub_df)):
                row.append(str(np.round(sub_df['Mean'][i],precision)) + '\n(' + str(np.round(sub_df['CI_lower'][i],precision)) + ', ' + str(np.round(sub_df['CI_upper'][i],precision)) + ')')
            #row.append(str(np.round(sub_df['Contrast'][0],2)))
            row.append(trend_symbol+'\n'+str(-1*np.round(sub_df['Contrast_CI_upper'][0],precision)) + '; ' + str(-1*np.round(sub_df['Contrast_CI_lower'][0],precision)))
        all_rows.append(row)

    df_final= pd.DataFrame(all_rows)

    df_final.to_csv('../results/associations/reshaped_association_table_model_'+str(ctr)+'.csv', index=False)
    

In [3]:
import pandas as pd

# Define the custom row order
row_order = [
    'bodyfat_percent',
    'bmi',
    'waistcirumference',
    'trig_log',
    'ldl',
    'hdl',
    'hba1c_percent',
]

# Function to add model number to row names
def add_model_number(df, model_number):
    df['original_row'] = df.index
    df['model'] = model_number
    df['row_name'] = df.index + f'_model{model_number}'
    return df.set_index('row_name')

# Read all CSV files
all_files = ['../results/associations/reshaped_association_table_model_'+str(i)+'.csv' for i in range(1,4)]

# Initialize an empty list to store dataframes
dfs = []

# Read each CSV file and add model number to row names
for i, filename in enumerate(all_files, 1):
    df = pd.read_csv(filename, index_col=0)
    df = add_model_number(df, i)
    dfs.append(df)

# Concatenate all dataframes
result = pd.concat(dfs)

# Create a categorical column for sorting based on the custom order
result['row_order'] = pd.Categorical(result['original_row'], categories=row_order, ordered=True)

# Sort the result by the custom row order, then by model number
result = result.sort_values(['row_order', 'model'])

# Remove the auxiliary columns used for sorting
result = result.drop(columns=['original_row', 'model', 'row_order'])

# Save the result to a new CSV file
result.to_csv('../results/associations/combined_association_tables.csv')
